In [1]:
import numpy as np
import re

In [2]:
# constants
BOARD_SIZE = 9
SECTOR_SIZE = 3

## Helper functions

In [49]:
def remove_value(arr, pattern):
    numpy_iterator = np.nditer(arr, flags=['multi_index', 'refs_ok'], op_flags=['readwrite'])
    for val in pattern:
        for cell in numpy_iterator:
            if str(cell) != pattern:
                cell[...] = str(cell).replace(val,'')

def get_row(arr, coord):
    return arr[coord[0],:]

def get_col(arr, coord):
    return arr[:,coord[1]]

def get_sector(arr, sector):
    row = sector // SECTOR_SIZE * SECTOR_SIZE
    col = SECTOR_SIZE * (sector % SECTOR_SIZE)
    return arr[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE]

def get_quad(arr, coord):
    x_home = coord[0] // SECTOR_SIZE * SECTOR_SIZE
    y_home = coord[1] // SECTOR_SIZE * SECTOR_SIZE
    return arr[x_home:x_home+3,y_home:y_home+3]

def update_neighbors(arr, coord, val):
    remove_value(get_row(arr, coord), val)
    remove_value(get_col(arr, coord), val)
    remove_value(get_quad(arr, coord), val)

def pretty_flat(board):
    width = np.max(get_length(board)) + 2
    line = '+' + '-'*(width*3) + '+' + '-'*(width*3) + '+' + '-'*(width*3) + '+'
    for idx, cell in enumerate(board):
        if not idx % 27:
            print(line)
        if not idx % 3:
            print('|', end='')
        print(F'{cell:^{width}s}', end='')
        if not ((idx + 1) % 9):
            print('|')
    print(line)
    
get_length = np.vectorize(len)

cell_contains = np.vectorize(lambda x, y : y in x)

---

### Load the puzzle

In [132]:
possible = np.full((9,9),'123456789')
board = np.full((9,9),'')

# hard
puzzle = re.sub('\W','0','2.......6.5..8..1...4...9...7.3.1......82.......7.5.3...9...4...8..1..5.6.......2')
# easy
puzzle = re.sub('\W','0','5..98.67.6......31.2.613.4..968.21.7..8..5.9.7.319....962.7..1.1.5...76..7.5..9..')
# medium
puzzle = re.sub('\W','0','.29.71..3..8...6..3...5....5.....97......4...4.75.8..1.6.42.3..2..9....6.916...52')
# hard
puzzle = re.sub('\W','0','..791.5....1.....3..9.4...2.4...83.....3.1....6..5...8.2..9...5...........4.8..7.')


for idx, value in enumerate(puzzle):
    if value != '0':
        row = idx // BOARD_SIZE
        col = idx - row * BOARD_SIZE
        board[row,col] = value
        possible[row,col] = value

solved = len(np.argwhere(get_length(np.array(list(re.sub('0','',puzzle))))==1))
print(F"Puzzle starts with {solved} solved locations.")

pretty_flat(board.flatten())

Puzzle starts with 23 solved locations.
+---------+---------+---------+
|       7 | 9  1    | 5       |
|       1 |         |       3 |
|       9 |    4    |       2 |
+---------+---------+---------+
|    4    |       8 | 3       |
|         | 3     1 |         |
|    6    |    5    |       8 |
+---------+---------+---------+
|    2    |    9    |       5 |
|         |         |         |
|       4 |    8    |    7    |
+---------+---------+---------+


---

### Step 1: Handle solved cells

Look at every cell that has only one **possible** value.

Run this cell multiple times and watch the puzzle change.

In [172]:
coordinate_list = np.argwhere(get_length(possible)==1)
for idc in coordinate_list:
    value = possible[idc[0], idc[1]]
    coord = (idc[0], idc[1])
    update_neighbors(possible, coord, value)
    board[coord] = value
    # empty this cell in the possible matrix as this cell has been solved
    possible[coord] = ''

solved = len(np.argwhere(get_length(board)==1))
print(F"Puzzle currently has {solved} solved locations.")

pretty_flat(board.flatten())

Puzzle currently has 79 solved locations.
+---------+---------+---------+
| 2  8  7 | 9  1  3 | 5  6  4 |
| 4  5  1 |    7    | 9  9  3 |
| 6  3  9 | 5  4  3 | 8  8  2 |
+---------+---------+---------+
| 1  4  5 | 7  2  8 | 3  9  6 |
| 9  9  8 | 3  6  1 | 2  5  4 |
| 7  6  2 | 4  5  9 | 7  1  8 |
+---------+---------+---------+
| 8  2  6 | 1  9  7 | 1  4  5 |
| 5  7  6 | 2  3  4 | 2  2  9 |
| 3  1  4 | 5  8  5 | 2  7  6 |
+---------+---------+---------+


### Step 2: Handle cells with matching possibilities

In [174]:
def get_common_patterns(arr):
    return_list = []
    pattern_dict = dict()
    numpy_iterator = np.nditer(arr, flags=['multi_index', 'refs_ok'], op_flags=['readwrite'])
    for cell in numpy_iterator:
        pattern_dict[str(cell)] = pattern_dict.get(str(cell),0) + 1
    for k,v in pattern_dict.items():
        if len(k) == v:
            return_list.append(k)
    return return_list

def remove_pattern_from_group(arr, patterns):
    for patt in patterns:
        numpy_iterator = np.nditer(arr, flags=['multi_index', 'refs_ok'], op_flags=['readwrite'])
        for cell in numpy_iterator:
            if str(cell) != patt:
                for val in list(patt):
                    cell[...] = str(cell).replace(val,'')

for row in range(BOARD_SIZE):
    patterns = get_common_patterns(possible[row,:])
    remove_pattern_from_group(possible[row,:], patterns)
for col in range(BOARD_SIZE):
    patterns = get_common_patterns(possible[:,col])
    remove_pattern_from_group(possible[:,col], patterns)
for row in range(0,BOARD_SIZE,3):
    for col in range(0,BOARD_SIZE,3):
        patterns = get_common_patterns(possible[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE])
        remove_pattern_from_group(possible[row:row+SECTOR_SIZE,col:col+SECTOR_SIZE], patterns)

pretty_flat(possible.flatten())
pretty_flat(board.flatten())

+------------+------------+------------+
|            |            |            |
|            | 68      26 |            |
|            |            |            |
+------------+------------+------------+
|            |            |            |
|            |            |            |
|            |            |            |
+------------+------------+------------+
|            |            |            |
|            |            |            |
|            |            |            |
+------------+------------+------------+
+---------+---------+---------+
| 2  8  7 | 9  1  3 | 5  6  4 |
| 4  5  1 |    7    | 9  9  3 |
| 6  3  9 | 5  4  3 | 8  8  2 |
+---------+---------+---------+
| 1  4  5 | 7  2  8 | 3  9  6 |
| 9  9  8 | 3  6  1 | 2  5  4 |
| 7  6  2 | 4  5  9 | 7  1  8 |
+---------+---------+---------+
| 8  2  6 | 1  9  7 | 1  4  5 |
| 5  7  6 | 2  3  4 | 2  2  9 |
| 3  1  4 | 5  8  5 | 2  7  6 |
+---------+---------+---------+


### Step 3: Handle hidden

In [175]:
for val in '123456789':
    # search every row
    for idx in range(BOARD_SIZE):
        if np.count_nonzero(cell_contains(possible[idx,:],val)) == 1:
            col = np.argwhere(cell_contains(possible[idx,:],val))[0]
            print(F"row {idx} is a match with {val} at [{idx},{col}]")
            update_neighbors(possible, (idx,col), val)
            board[idx,col] = val

    # search every column
    for idx in range(BOARD_SIZE):
        if np.count_nonzero(cell_contains(possible[:,idx],val)) == 1:
            row = np.argwhere(cell_contains(possible[:,idx],val))[0][0]
            print(F"column {idx} is a match with {val} at [{row},{idx}]")
            update_neighbors(possible, (row,idx), val)
            board[row,idx] = val

    # search every sector
    for idx in range(BOARD_SIZE):
        if np.count_nonzero(cell_contains(get_sector(possible,idx),val)) == 1:
            coords = np.argwhere(cell_contains(get_sector(possible,idx),val))
            row = coords[0][0] + SECTOR_SIZE * (idx // SECTOR_SIZE)
            col = coords[0][1] + SECTOR_SIZE * (idx % SECTOR_SIZE)
            print(F"sector {idx} is a match with {val} at [{row},{col}]")
            update_neighbors(possible, (row,col), val)
            board[row,col] = val


pretty_flat(possible.flatten())
pretty_flat(board.flatten())

row 1 is a match with 2 at [1,[5]]


TypeError: only integer scalar arrays can be converted to a scalar index